# Aprendizaje de métricas distancia

La **"distancia semántica"** entre un par de puntos, $d(\boldsymbol{x},\boldsymbol{x}')\in\mathbb{R}^{\geq 0}$ para $\boldsymbol{x},\boldsymbol{x}'\in\mathcal{X}$, o equivalentemente su similtud $s(\boldsymbol{x},\boldsymbol{x}')\in\mathbb{R}^{\geq 0}$, es un concepto muy importante en clasificación KNN, aprendizaje auto-supervisado, clustering, recuperación de información, seguimiento visual, etc.

Si $\mathcal{X}=\mathbb{R}^D$ y la dimensión $D$ no es muy alta, se suele emplear la distancia de Mahalanobis:
$$d(\boldsymbol{x},\boldsymbol{x}')=\sqrt{(\boldsymbol{x}-\boldsymbol{x}')^tM(\boldsymbol{x}-\boldsymbol{x}')}$$
donde $M$ se aprende directamente o, indirectamente, a través de una proyección lineal.

En el caso de entradas de alta dimensión, se aprende un embedding $\boldsymbol{e}=f(\boldsymbol{x})$ con una red neuronal profunda y luego se calculan distancias en el espacio de embedding. Se pretende que dichas distancias sean altamente significativas y menos sujetas a la maldición de la dimensionalidad. Este caso, conocido como **deep metric learning,** ha recibido gran atención en los últimos años.

## Deep metric learning (DML)

Sea $\boldsymbol{e}=f(\boldsymbol{x};\boldsymbol{\theta})\in\mathbb{R}^L$ un embedding de la entrada en un espacio "semántico" de baja dimensión y sea $\hat{\boldsymbol{e}}=\boldsymbol{e}/\lVert\boldsymbol{e}\rVert_2$ su versión $\ell_2$-normalizada. La $\ell_2$-normalización garantiza que todos los puntos se hallen en la hiperesfera unitaria. Asumimos que la comparación entre dos puntos se realizará mediante la **distancia Euclídea normalizada** (con mayor similitud cuanto menor sea),
$$d(\boldsymbol{x}_i,\boldsymbol{x}_j;\boldsymbol{\theta})%
=\lVert\hat{\boldsymbol{e}}_i-\hat{\boldsymbol{e}}_j\rVert_2^2%
=(\hat{\boldsymbol{e}}_i-\hat{\boldsymbol{e}}_j)^t(\hat{\boldsymbol{e}}_i-\hat{\boldsymbol{e}}_j)%
=2-2\hat{\boldsymbol{e}}_i^t\hat{\boldsymbol{e}}_j$$
o, equivalentemente, mediante la **similitud coseno** (con mayor similitud cuanto menor sea):
$$d(\boldsymbol{x}_i,\boldsymbol{x}_j;\boldsymbol{\theta})%
=\hat{\boldsymbol{e}}_i^t\hat{\boldsymbol{e}}_j$$

DML debe aprender una función de embedding tal que los ejemplos similares estén más cerca que los ejemplos no similares. Más precisamente, supongamos que $\mathcal{D}=\{(\boldsymbol{x}_i,y_i):i=1:N\}$ es un conjunto de datos etiquetados y que $S=\{(i,j):y_i=y_j\}$ es un conjunto de pares similares derivado. La propiedad básica que se desea cumplir es que, **si $(i,j)\in S$ pero $(i,k)\not\in S$, entonces $\boldsymbol{x}_i$ y $\boldsymbol{x}_j$ deben estar cerca en el espacio de embedding, pero $\boldsymbol{x}_i$ y $\boldsymbol{x}_k$ deben estar lejos.** Se han propuesto diversos criterios para cumplir esta propiedad; también con datos no etiquetados si existe una manera de definir pares similares. 

## Pérdidas de clasificación

Si disponemos de datos etiquetados de C clases, podemos ajustar un modelo de clasificación con coste temporal $O(NC)$ y reutilizar caracterı́sticas ocultas como función de embedding. En general, esta aproximación sencilla y escalable, aunque solo embebe ejemplos en el lado correcto de una frontera de decisión, lo que no garantiza que ejemplos similares estén cerca y diferentes lejos. Otro incoveniente es que no vale para datos no etiquetados.

## Pérdidas de ranking

Las pérdidas de ranking tratan de garantizar que ejemplos similares estén más cerca que ejemplos diferentes. La mayorı́a de métodos propuestos no requieren etiquetas de clase.

### Pérdida (contrastiva) de pares y redes siamesas

Una de las primeras aproximaciones al aprendizaje de representaciones a partir de pares similares/diferentes se basa en la minimización de la **pérdida contrastiva de pares:**
$$\mathcal{L}(\boldsymbol{\theta};\boldsymbol{x}_i,\boldsymbol{x}_j)%
=\mathbb{I}(y_i=y_j)d(\boldsymbol{x}_i,\boldsymbol{x}_j)^2%
+\mathbb{I}(y_i\neq y_j)[m-d(\boldsymbol{x}_i,\boldsymbol{x}_j)]_+^2$$
donde $[z]_+=\max(z, 0)$ es la pérdida bisagra y $m>0$ es un margen. La minimización se realiza sobre todos los pares de datos. En general, queremos que los pares positivos (con la misma etiqueta) estén cerca y que los pares negativos (con distintas etiquetas) estén alejados con al menos un margen de seguridad mı́nimo.

La **red siamesa** minimiza la pérdida de pares usando el mismo extractor de caracterı́sticas para ambas entradas, $\boldsymbol{x}_i$ y $\boldsymbol{x}_j$, al calcular la distancia:
<p align="center"><img width="500" src="Figure_16.5_A.png"></p>

La principal desventaja de la pérdida de pares es que la optimización de pares positivos es independiente de la de negativos, lo que dificulta la comparación de sus magnitudes.

### Pérdida triplete

La **pérdida triplete** se propone como solución a la principal desventaja de la pérdida de pares, esto es, para que la optimización de pares positivos y negativos sea interdependiente, no independiente. Para ello, por cada ejemplo o **ancla** $\boldsymbol{x}_i$, buscamos un ejemplo similar (positivo) $\boldsymbol{x}_i^+$ y otro diferente (negativo) $\boldsymbol{x}_i^-$, y minimizamos la siguiente pérdida, promediada sobre todos los tripletes:
$$\mathcal{L}(\boldsymbol{\theta};\boldsymbol{x}_i,\boldsymbol{x}_i^+,\boldsymbol{x}_i^-)%
=[d(\boldsymbol{x}_i,\boldsymbol{x}_i^+)^2-d(\boldsymbol{x}_i,\boldsymbol{x}_i^-)^2+m]_+$$
La pérdida triplete persigue que la distancia de la ancla al positivo sea menor que al negativo, en al menos un margen de seguridad $m$. El coste temporal de su minimización naíf es $O(N^3)$. En la práctica, esta pérdida se minimiza una **red triplete** entrenada con minibatches que incluyan al menos un ejemplo positivo y otro negativo de cada ancla:
<p align="center"><img width="500" src="Figure_16.5_B.png"></p>